In [130]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

%matplotlib inline

In [131]:
dataset = '../datasets/Boston.csv'
data = pd.read_csv(dataset, index_col=0)
data.head()

,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [132]:
from sklearn.linear_model import LinearRegression
import sklearn.metrics as metrics

In [133]:
lstat_aug = pd.concat([data.lstat, pd.Series(np.ones(data.lstat.shape), index=data.lstat.index, name='intercept')], axis=1)

In [134]:
medv_on_lstat = LinearRegression().fit(data.lstat.values.reshape(-1, 1), data.medv)
medv_on_lstat.coef_, medv_on_lstat.intercept_

(array([-0.95004935]), 34.5538408793831)

In [135]:
def add_intercept_term(X):
    if isinstance(X, (pd.DataFrame, pd.Series)):
        return pd.concat([X, pd.Series(np.ones(data.lstat.shape), index=X.index, name='intercept')], axis=1)
    elif isinstance(X, np.ndarray):
        return pd.concat([pd.DataFrame(X, columns=[f'X{i+1}' for i in range(X.shape[1])]), pd.Series(np.ones(X.shape[0]), name='intercept')], axis=1)
    else:
        return pd.concat([pd.Series(X, name='X1'), pd.Series(np.ones(len(X)), name='intercept')], axis=1)

In [136]:
def report_model(model, X, y):
    results = {}
    pred_y = model.predict(X)
    residuals = y - pred_y
    residual_stats = pd.Series(residuals).describe()
    results['Residual stats'] = residual_stats
    
    RSS = residuals.T @ residuals
    results['RSS'] = RSS
    
    resid_from_mean = y - y.mean()
    TSS = resid_from_mean.T @ resid_from_mean
    results['TSS'] = TSS
    
    R_squared = 1 - RSS / TSS
    results['R squared'] = R_squared
    
    F_score = ((TSS - RSS) / (X.shape[1])) / (RSS / (X.shape[0] - X.shape[1] - 1))
    results['F_score'] = F_score
    
    sigma_squared_hat = RSS / (X.shape[0] - X.shape[1] - 1)
    results['Sigma squared estimation'] = sigma_squared_hat
    
    RSE = sigma_squared_hat ** 0.5
    results['RSE'] = RSE
    
    aug_X = add_intercept_term(X)
    var_beta_hat = np.linalg.inv(aug_X.T @ aug_X) * sigma_squared_hat
    results['Betha variance estimation'] = var_beta_hat
    
    se = []
    for p_ in range(X.shape[1] + 1):
        standard_error = var_beta_hat[p_, p_] ** 0.5
        se.append(standard_error)
        results[f"SE(beta_hat_{p_})"] = standard_error
    [print(f'{k}: {v}') for k, v in results.items()]
    return results

In [137]:
results = report_model(medv_on_lstat, data.lstat.values.reshape(-1, 1), data.medv)

Residual stats: count    5.060000e+02
mean    -5.673108e-15
std      6.209603e+00
min     -1.516745e+01
25%     -3.989612e+00
50%     -1.318186e+00
75%      2.033701e+00
max      2.450013e+01
Name: medv, dtype: float64
RSS: 19472.381418326433
TSS: 42716.29541501976
R squared: 0.5441462975864799
F_score: 601.6178711098956
Sigma squared estimation: 38.63567741731435
RSE: 6.2157604053980675
Betha variance estimation: [[ 0.00150028 -0.01898311]
 [-0.01898311  0.31654954]]
SE(beta_hat_0): 0.038733416212639364
SE(beta_hat_1): 0.5626273549884322


In [158]:
def intervals(model, X, model_stats):
    if isinstance(X, (float, int)):
        X = np.array(X).reshape(-1, 1)
    t = 1.9647
    results = {}

    predictions = model.predict(X)
    aug_X = add_intercept_term(X)
    
    cov = model_stats['Betha variance estimation']
    model_var = model_stats['Sigma squared estimation']
    
    se = (aug_X * np.dot(cov, aug_X.T).T).sum(1)
    
    CI = t * se ** 0.5
    PI = t * (se + model_var) ** 0.5
    results['Confidence intervals'] = np.array([predictions - CI, predictions + CI]).T
    results['Prediction intervals'] = np.array([predictions - PI, predictions + PI]).T
    
    [print(f'{k}: \n{v}') for k, v in results.items()]
    return results

In [159]:
predictions = intervals(medv_on_lstat, np.array([5, 10, 15]).reshape(-1, 1), results)

Confidence intervals: 
[[29.00740464 30.59978358]
 [24.47412671 25.63256797]
 [19.73158292 20.87461823]]
Prediction intervals: 
[[17.56556268 42.04162554]
 [12.82751436 37.27918032]
 [ 8.07763011 32.52857103]]


In [160]:
y = data.medv
X = data.lstat.values.reshape(-1, 1)
y_pred = medv_on_lstat.predict(X)

In [141]:
import statsmodels.api as sm

In [142]:
mod = sm.OLS(y, sm.add_constant(X)).fit()
mod.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                   medv   R-squared:                       0.544
Model:                            OLS   Adj. R-squared:                  0.543
Method:                 Least Squares   F-statistic:                     601.6
Date:                Fri, 28 Aug 2020   Prob (F-statistic):           5.08e-88
Time:                        18:18:34   Log-Likelihood:                -1641.5
No. Observations:                 506   AIC:                             3287.
Df Residuals:                     504   BIC:                             3295.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         34.5538      0.563     61.415      0.000      33.448      35.659
x1            -0.9500      0.039    -24.528      0.000      -1.026      -0.874
==============================================================================
Omnibus:                      137.043   Durbin-Watson:                   0.892
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              291.373
Skew:                           1.453   Prob(JB):                     5.36e-64
Kurtosis:                       5.319   Cond. No.                         29.7
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [144]:
res = mod.get_prediction(sm.add_constant([5, 10, 15]))
res.summary_frame()

,mean,mean_se,mean_ci_lower,mean_ci_upper,obs_ci_lower,obs_ci_upper
0,29.803594,0.405247,29.007412,30.599776,17.565675,42.041513
1,25.053347,0.294814,24.474132,25.632563,12.827626,37.279068
2,20.303101,0.290893,19.731588,20.874613,8.077742,32.528459
